<a href="https://colab.research.google.com/github/hirenvadher954/Awesome_Machine_Learning/blob/master/FruitsAndVegetables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! Required Kaggle Api Json to Download Database

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d moltean/fruits

 98% 744M/760M [00:09<00:00, 53.0MB/s]
100% 760M/760M [00:09<00:00, 80.5MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fruits.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [4]:
import tensorflow as tf
import os   

In [6]:
base_dir = '/content/files/fruits-360/Training'
train_dir = '/content/files/fruits-360/Test'


In [7]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
 
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset='training',
    
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset='validation',
    
)



Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.


In [8]:

labels = "\n".join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
  f.write(labels)


In [9]:
Img_Shape = (IMAGE_SIZE,IMAGE_SIZE,3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=Img_Shape,
    include_top=False,
    weights = 'imagenet'
)

base_model.trainable = False;

9412608/9406464 [==============================] - 0s 0us/step


In [11]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131,activation='softmax')

])

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    )

Epoch 1/10
847/847 [==============================] - 98s 116ms/step - loss: 0.6685 - accuracy: 0.8483 - val_loss: 0.1714 - val_accuracy: 0.9512
Epoch 2/10
847/847 [==============================] - 95s 112ms/step - loss: 0.0298 - accuracy: 0.9941 - val_loss: 0.1212 - val_accuracy: 0.9637
Epoch 3/10
847/847 [==============================] - 92s 108ms/step - loss: 0.0202 - accuracy: 0.9948 - val_loss: 0.1803 - val_accuracy: 0.9450
Epoch 4/10
847/847 [==============================] - 90s 106ms/step - loss: 0.0193 - accuracy: 0.9941 - val_loss: 0.2473 - val_accuracy: 0.9345
Epoch 5/10
847/847 [==============================] - 88s 104ms/step - loss: 0.0153 - accuracy: 0.9956 - val_loss: 0.2638 - val_accuracy: 0.9376
Epoch 6/10
847/847 [==============================] - 88s 104ms/step - loss: 0.0174 - accuracy: 0.9943 - val_loss: 0.1816 - val_accuracy: 0.9512
Epoch 7/10
847/847 [==============================] - 88s 104ms/step - loss: 0.0143 - accuracy: 0.9954 - val_loss: 0.1491 - val_ac

In [17]:
save_model_dir = ''
tf.saved_model.save(model,save_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(save_model_dir)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets
